### steps
1. Create an sample SQL database
2. create config.ini for Mysql database connection
3. create table
4. performing SQL window on database to solve department highest salary


In [2]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [3]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """
    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(query_insert, table_values):
    """execute query to insert values to table
    :query_table:: to create scheme table
    :table_values:: to be inserted table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('Inserting values to designed table...')
        cursor.executemany(query_insert, table_values)

        conn.commit()
        print('Values inserted to designeed table')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

In [4]:
def main():
    query_create_employee = "CREATE TABLE Employee1 (Id INT, \
                                                    Name VARCHAR(255), \
                                                    Salary INT, \
                                                    DepartmentId INT, \
                                                    PRIMARY KEY (Id), \
                                                    FOREIGN KEY (DepartmentId) REFERENCES Department(Id) \
                                                    );"
    
    query_create_department = "CREATE TABLE Department1 (Id INT, \
                                                        Name VARCHAR(255), \
                                                        PRIMARY KEY (Id) \
                                                        );"
    
    query_insert_employee = "INSERT INTO Employee1 (Id,Name,Salary,DepartmentId) VALUES(%s,%s,%s,%s)"
    query_insert_department = "INSERT INTO Department1 (Id,Name) VALUES(%s,%s)"
    ###############
    employee = [(1, 'Joe', 60000, 1), \
                (2, 'Sam', 50000, 1), \
                (4, 'Max', 50000, 2)
                ]
    
    department = [(1, 'IT'),
                (2, "HR")
                ]
    
    create_msql_table(query_create_department)
    create_msql_table(query_create_employee)
    insert_msql_table(query_insert_department, department)
    insert_msql_table(query_insert_employee, employee)

if __name__ == '__main__':
    main()

Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'department1' already exists
Connecting to MYSQL database...
Connected to MYSQL database
Creating Mysql table...
Error:  1050 (42S01): Table 'employee1' already exists
Connecting to MYSQL database...
Connected to MYSQL database
Inserting values to designed table...
Error:  1062 (23000): Duplicate entry '1' for key 'department1.PRIMARY'
Connecting to MYSQL database...
Connected to MYSQL database
Inserting values to designed table...
Error:  1062 (23000): Duplicate entry '1' for key 'employee1.PRIMARY'


In [5]:
query_fetching_highest_salary = " \
                                    select * \
                                    from ( \
                                            select RANK() OVER (PARTITION BY D.Name ORDER BY Salary DESC) as SalaryRank, \
                                                    E.Id as EmployeeId, E.Name as Employee, E.Salary, E.departmentId, D.Id as DepId, D.Name as Department\
                                            from Employee1 E, Department1 D \
                                            where E.DepartmentId = D.Id \
                                            ) r \
                                    where SalaryRank = 1 \
                                    "

try:
    dbconfig = read_db_config()
    conn = MySQLConnection(**dbconfig)
    cursor = conn.cursor()
    cursor.execute(query_fetching_highest_salary)

    row = cursor.fetchone()

    while row is not None:
        print(row)
        row = cursor.fetchone()

except Error as e:
    print(e)

finally:
    cursor.close()
    conn.close()

(1, 4, 'Max', 50000, 2, 2, 'HR')
(1, 1, 'Joe', 60000, 1, 1, 'IT')


In [6]:
query_fetching = "select Department, Em.Name as Employee, Em.Salary \
                from Employee1 Em, ( \
                                    select D.name as Department, max(E.Salary) as max \
                                    from Employee1 E, Department1 D \
                                    where E.DepartmentId = D.Id \
                                    group by 1 ) Rnk \
                where Em.Salary = Rnk.max \
                "

In [7]:
query_fetching1 = " select DISTINCT E.Name AS Employee, D.Name as Department, E.Salary \
                    from Employee1 E, Department1 D, ( \
                                                    select DepartmentId as dep_id, max(Salary) as max \
                                                    from Employee1 \
                                                    group by dep_id ) Rnk \
                    where E.DepartmentId=D.Id and E.Salary=Rnk.max \
                "

In [8]:
!mysql -u root --password=root python_mysql -e "{query_fetching1}"

mysql: [Warning] Using a password on the command line interface can be insecure.
+----------+------------+--------+
| Employee | Department | Salary |
+----------+------------+--------+
| Joe      | IT         |  60000 |
| Sam      | IT         |  50000 |
| Max      | HR         |  50000 |
+----------+------------+--------+
